In [23]:
import pandas as pd
import spacy
import time
import math
import re

class Sentiment_Analysis():
    def __init__(self,alpha = 1.0):
        self.alpha = alpha
        self.nlp = spacy.load("en_core_web_sm")
        self.Class_Probability = [0,0,0,0,0,0]
        self.Class = {
            "fear" : 0,
            "sad" : 1,
            "anger" : 2,
            "love" : 3,
            "suprise" : 4,
            "joy" :5
        }
        self.Vocabulary = {}
        self.StopWords = ["i", "","me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
            "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its",
            "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
            "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having",
            "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while","of", "at", "by", "for", "with", 
            "about","between", "into", "through", "during", "before", "after", "to", "from", "up", "down", "in", "out", "over", "under", "again",
            "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
            "few", "more", "most", "other", "some", "such", "nor", "only", "own", "same", "so",
            "than", "can", "will", "just", "now","do"]

    def Process(self,current, end,rate):
        percentage = 100 * (current/end)
        bar = "#" * int(percentage/2) + "-" * (50 - int(percentage/2))

        print(f"\r| {bar} | , {percentage: .2f} , {rate: .2f}",end="\r")

    def Train(self,Data,max_count):
        count = 0
        for ind,line in Data.iterrows():
            start = time.perf_counter()
            words = self.Tokeniser(line[0])
            class_index = self.Class[line[1]]
            for word in words:
                if word not in self.Vocabulary:
                    self.Vocabulary[word] = [0,0,0,0,0,0]
                
                (self.Vocabulary[word])[class_index]+=1
                self.Class_Probability[class_index]+=1
            end = time.perf_counter()
            count+=1
            self.Process(count,max_count,(end-start)*(max_count-count))
            if(count == max_count):
                break
        V = len(self.Vocabulary)

        for i in range(0,6):
            for word in self.Vocabulary:
                (self.Vocabulary[word])[i] = (((self.Vocabulary[word])[i] + self.alpha)/(self.Class_Probability[i] + (V*(self.alpha))))

        tmp = sum(self.Class_Probability)
        for i in range(0,6):
            self.Class_Probability[i] = (self.Class_Probability[i])/tmp


    def Test(self,TestData,max_count):
        score = 0
        count = 0
        for ind,text in TestData.iterrows():
            start = time.perf_counter()
            words = self.Tokeniser(text[0])

            Scores = [0]*6
            for i in range(0,6):
                Scores[i] = math.log(self.Class_Probability[i])

            for word in words:
                if word in self.Vocabulary:
                    word_probability = self.Vocabulary[word]
                    for i in range(0,6):
                        Scores[i] += (math.log(word_probability[i]))

    
            if self.Class[text[1]] == Scores.index(max(Scores)):
                score+=1

            count+=1
            end = time.perf_counter()
            self.Process(count,max_count,(end-start)*(max_count-count))

            if count == max_count:
                break
        
        return (100 * (score/max_count))
    

    def Predit(self,text):
        words = self.Tokeniser(text)
        Score = [0,0,0,0,0,0]
        for i in range(0,6):
            Score[i] = self.Class_Probability[i]

        for word in words:
            if word in self.Vocabulary:
                for i in range(0,6):
                    Score[i] += math.log((self.Vocabulary[word])[i])

        return Score
            
                       
    def lemmatization(self,word):
        return self.nlp(word)[0].lemma_

    def Tokeniser(self,text):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = list(set(text.split(" ")))
        text = [self.lemmatization(tex) for tex in text if tex not in self.StopWords]
        return text

a = Sentiment_Analysis(1.0)


In [ ]:
DataSet = pd.read_csv("train.csv")

a.Train(DataSet,317058)

C:\Users\NEXT GEN\AppData\Local\Temp\ipykernel_13968\4128354281.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  words = self.Tokeniser(line[0])
C:\Users\NEXT GEN\AppData\Local\Temp\ipykernel_13968\4128354281.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  class_index = self.Class[line[1]]


In [35]:
Test = pd.read_csv("test.csv")

print(f"\n {a.Test(Test,100)} \n")

C:\Users\NEXT GEN\AppData\Local\Temp\ipykernel_13968\4128354281.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  words = self.Tokeniser(text[0])
C:\Users\NEXT GEN\AppData\Local\Temp\ipykernel_13968\4128354281.py:83: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if self.Class[text[1]] == Scores.index(max(Scores)):


| ################################################## | ,  100.00 ,  0.00
 90.0 



In [36]:

sc = a.Predit("ima hate you")

ind = sc.index(max(sc))


if ind == 0:
    print("fear")
elif ind == 1:
    print("sad")
elif ind == 2:
    print("anger")
elif ind == 3:
    print("love")
elif ind == 4:
    print("suprise")
else:
    print("joy")

anger


In [34]:
import pickle

with open("model.pkl","wb") as f:
    pickle.dump(a,f)